In [6]:
import gym
import numpy as np
import pandas as pd
from gym import spaces

In [7]:
class StockTradingEnv(gym.Env):
    def __init__(self, data):
        super(StockTradingEnv, self).__init__()
        self.data = data.reset_index(drop=True)
        self.n_features = self.data.shape[1] - 1  # Exclude the target column
        self.current_step = 0
        self.cash = 10000  # Starting cash
        self.stocks_held = 0  # Shares 
        self.total_shares = 0

        # Define action space: 0 = Hold, 1 = Buy, 2 = Sell
        self.action_space = spaces.Discrete(3)

        # Observation space: Features from the dataset
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.n_features,), dtype=np.float32
        )

    def reset(self):
        """Reset the environment to start a new episode."""
        self.current_step = 0
        self.cash = 10000
        self.stocks_held = 0
        self.total_shares = 0
        return self.data.iloc[self.current_step, 1:].values  # Exclude 'date'

    def step(self, action):
        """Perform an action and return the next state, reward, and done flag."""
        current_price = self.data.iloc[self.current_step]['PRC']
        reward = 0

        # Execute action
        if action == 1:  # Buy
            if self.cash >= current_price:
                self.cash -= current_price
                self.stocks_held += 1
        elif action == 2:  # Sell
            if self.stocks_held > 0:
                self.cash += current_price
                self.stocks_held -= 1

        # Calculate reward as the change in portfolio value
        portfolio_value = self.cash + self.stocks_held * current_price
        reward = portfolio_value - 10000  # Compare with initial cash

        # Increment step
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1

        # Get the next state
        next_state = self.data.iloc[self.current_step, 1:].values

        return next_state, reward, done, {}

    def render(self):
        """Render the current state of the environment."""
        print(
            f"Step: {self.current_step}, Cash: {self.cash}, Stocks Held: {self.stocks_held}"
        )

# Load your feature-engineered data
data = pd.read_csv('/Users/amulya/Desktop/Capstone/DSCI-601-Amy/Data/FeatureEngineered/AKAM_feature_engineeredv2.csv')

# Initialize the environment
env = StockTradingEnv(data)

In [8]:
pip install stable-baselines3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 12.0 MB/s eta 0:00:0000:0100:01
  Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.8 MB/s eta 0:00:0000:0100:01
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached fonttools-4.54.1-cp310-cp310-macosx_11_0_arm64.whl (2.3 MB)
  Using cached kiwisolver-1.4.7-cp310-cp310-macosx_11_0_arm64.whl (64 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━